In [294]:
#Import Necessary Modules
import os
import sys
import re
import time
import shutil
import collections
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import KFold
import random

# Stage 2 Clean

In [352]:
newpath = '../../DataforProject/reviews/Yuying_tagged/'

def exampleGen(newpath):
    # max number of words to get before and after the title
    MAXWORDS = 3
    # max number of negtive examples from each document
    MAXNEG = 10
    # max number of words in negtive examples
    MAXNEGWORDS = 3

    # output list of examples for feature generation
    # use this as the input of feature generation
    # format: (title, wordsbefore, wordsafter, class)
    outlist = []
    exid = 0
    ouputfile = open("examples.txt",'w')

    pos_num = 0
    neg_num = 0
    badfile_num = 0
    file_num = 0

    # path to the html folder
    #newpath = './tagged/'
    
    files = os.listdir(newpath)
    for file in files:
        with open(newpath+file) as f:
            #print(file)
            #ouputfile.write(file+'\n')

            file_num += 1
            # wrap all lines without new line charactar
            lines=""
            for line in f:
                lines += line.strip()
            words = lines.split(" ")

            title = ""
            within = False
            wordsprocessed = []
            badfile = False
            for i in range(len(words)):
                tmplist = []
                word = words[i]
                if word == '': continue
                #print(word)
                pos1 = word.find("<myy>")
                pos2 = word.find("</myy>")
                # if no tag found
                if pos1 == -1 and pos2 == -1:
                    wordsprocessed.append((word,within))
                # if both are found
                if pos1 >= 0 and pos2 >= 0:
                    tmplist = word.split("<myy>")
                    if len(tmplist)>2 or pos2 < pos1:
                        badfile = True
                        break
                    if tmplist[0]!='':
                        wordsprocessed.append((tmplist[0],within))
                    within = True
                    tmplist = tmplist[1].split("</myy>")
                    wordsprocessed.append((tmplist[0],within))
                    within = False
                    if tmplist[1]!='':
                        wordsprocessed.append((tmplist[1],within))
                    continue
                # if only start tag
                if pos1 >= 0:
                    tmplist = word.split("<myy>")
                    if len(tmplist)>2:
                        badfile = True
                        break
                    if tmplist[0]!='':
                        wordsprocessed.append((tmplist[0],within))
                    within = True
                    wordsprocessed.append((tmplist[1],within))
                    continue
                # if only end tag
                if pos2 >= 0:
                    tmplist = word.split("</myy>")
                    if len(tmplist)>2:
                        badfile = True
                        break
                    wordsprocessed.append((tmplist[0],within))
                    within = False
                    if tmplist[1]!='':
                        wordsprocessed.append((tmplist[1],within))

            if badfile is True:
                badfile_num += 1
                continue

            # get positive examples from the processed words
            wordsbefore = []
            wordsafter = []
            title = ""
            for i in range(len(wordsprocessed)):
                item = wordsprocessed[i]
                if item[1] is True:
                    # if it is the first word
                    if i == 0:
                        title += item[0]
                        while len(wordsbefore) < MAXWORDS:
                            wordsbefore.append("")
                    # if word before is also title
                    elif wordsprocessed[i-1][1]:
                        title += " " + item[0]
                    else:
                        title += item[0]
                        # get words before the title
                        for j in range(i-1,i-1-MAXWORDS,-1):
                            if j >= 0:
                                wordsbefore.append(wordsprocessed[j][0])
                            else:
                                # add empty element
                                while len(wordsbefore) < MAXWORDS:
                                    wordsbefore.append("")
                                break
                else:
                    # if the word before this is title
                    if i != 0 and wordsprocessed[i-1][1]:
                        # get words after the title
                        for j in range(i,i+MAXWORDS):
                            if j < len(wordsprocessed):
                                wordsafter.append(wordsprocessed[j][0])
                            else:
                                # add empty element
                                while len(wordsafter) < MAXWORDS:
                                    wordsafter.append("")
                                break
                        # add title and words before and after to output list
                        featuretuple = (exid,title, wordsbefore, wordsafter, True)
                        #print(featuretuple)
                        outlist.append(featuretuple)
                        exid += 1

                        # generate output string
                        # this part can be skipped if only want features
                        outstr = title + ", ["
                        for w in wordsbefore:
                            if w is None:
                                outstr += 'None '
                            else:
                                outstr += w + ' '
                        outstr += '], ['
                        for w in wordsafter:
                            if w is None:
                                outstr += 'None '
                            else:
                                outstr += w + ' '
                        outstr += '], T\n'
                        ouputfile.write(outstr)

                        pos_num += 1

                        title = ""
                        wordsbefore = []
                        wordsafter = []

            # get negtive examples
            wordsbefore = []
            wordsafter = []
            title = ""
            count = 0
            poslist = []
            # skip the document if there are not enough words in it
            if len(wordsprocessed) < 100:
                continue
            while count < MAXNEG:
                pos = random.randint(MAXWORDS+1, len(wordsprocessed)-MAXWORDS-MAXNEGWORDS)
                title_len = random.randint(1,MAXNEGWORDS)
                # get neg title
                alltrue = True
                for i in range(title_len):
                    if wordsprocessed[pos+i][1] is False:
                        alltrue = False
                    title += wordsprocessed[pos+i][0] + " "
                # if all words are in title
                if alltrue is True:
                    title = ""
                    continue
                title = title.strip()
                # get words before and after
                for i in range(0,MAXWORDS):
                    wordsbefore.append(wordsprocessed[pos-i-1][0])
                    wordsafter.append(wordsprocessed[pos+title_len+i][0])
                featuretuple = (exid,title, wordsbefore, wordsafter, False)
                #print(featuretuple)
                outlist.append(featuretuple)
                exid += 1

                # generate output string
                # this part can be skipped if only want features
                outstr = title + ", ["
                for w in wordsbefore:
                    if w is None:
                        outstr += 'None '
                    else:
                        outstr += w + ' '
                outstr += '], ['
                for w in wordsafter:
                    if w is None:
                        outstr += 'None '
                    else:
                        outstr += w + ' '
                outstr += '], F\n'
                ouputfile.write(outstr)


                neg_num += 1
                count += 1
                title=""
                wordsbefore = []
                wordsafter = []

    ouputfile.close()
    ##print("number of pos examples:", pos_num)
    #print("number of neg examples:", neg_num)
    #print("number of bad files:", badfile_num)
    #print(file_num)
    return outlist



In [353]:
dat= exampleGen(newpath)

In [356]:
dat[14]

(14,
 'as her',
 ['Byington', 'Spring', 'and'],
 ['mother.', 'The', 'newest'],
 False)

In [143]:
#Test Set
test1 = ('HAMLET',["Mann's", 'Laurie','1991'], 
        ['is', 'a', 'superbly-produced'], True)
test2 = ('HAM',["Am'd", 'film','1998'], 
        ['isss', 'aaa', '2001'], True)
test3 = ('Ham',["Am's", 'Laur',' (1998) '], 
        ['isss', "aaa'd", 'super'], False)
test4 = ('HAM 2',["'", 'Laur','198'], 
        ['isss', 'aaa', 'super'], True)
test5 = ('HAM ',['Am', 'review','test'], 
        ['isss', '*', 'super'],True)
test6 = ('HAM ',['Am', 'Laur','test'], 
        ['isss', 'II', 'super'],True)
test7 = ('HAM ',[')', 'Laur','test'], 
        ['isss', 'The', 'super'],True)
test8 = ('BATMANT',["nice", '(','1991'], 
        ['is', 'a', 'superbly'], True)
test9 = ('Batman',["Am'd", 'review','1998'], 
        ['isss', 'aaa', '2001'], True)
test10 = ('Batman',["Am's", '"',' (1998) '], 
        ['isss', "aaa'd", 'super'], False)



cases = [test1,test2,test3, test4, test5, test6, test7, test8, test9, test10]

In [40]:
cases[0]

('HAMLET',
 ["Mann's", 'Laurie', '1991'],
 ['is', 'a', 'superbly-produced'],
 True)

In [314]:
#%run ./gen_ex.py

In [257]:
dat = []
for line in open('examples.txt'):
  dat.append(tuple(line.strip().split(',')))
dat[1]

('LITTLE WOMEN', " [1868 Alcott's LousiaMay ]", ' [', ' based in ]', ' T')

# Pre-Processing

In [53]:
dat = []
for line in open('examples.txt'):
  dat.append(tuple(line.strip().split(',')))
dat[1]

('LITTLE WOMEN', " [1868 Alcott's LousiaMay ]", ' [', ' based in ]', ' T')

In [118]:
cases[0]

('HAMLET',
 ["Mann's", 'Laurie', '1991'],
 ['is', 'a', 'superbly-produced'],
 True)

In [70]:
dat[0]

[('LITTLE WOMEN', ' [None None None ]', ' [A film review ]', ' T'),
 ('LITTLE WOMEN', " [1868 Alcott's LousiaMay ]", ' [', ' based in ]', ' T'),
 ('MY BRILLIANT CAREER',
  ' [of Armstrong Gillian ]',
  ' [and MRS. SOFFEL ]',
  ' T'),
 ('MRS. SOFFEL', ' [and CAREER BRILLIANT ]', ' [. The Marches ]', ' T'),
 ('INTERVIEW WITH THE VAMPIRE', ' [from child a ]', ' [. Among other ]', ' T'),
 ('LITTLE WOMEN',
  " [And HEAVENLYCREATURES. Jackson's ]",
  ' [sorely needs something ]',
  ' T'),
 ('LITTLE WOMEN', ' [give I warmfeeling. ]', ' [a high +1 ]', ' T'),
 ('and the', ' [version this word--in ]', ' [only one for ]', ' F'),
 ('tightly-knit family of',
  ' [very a are ]',
  ' [four sisters andtheir ]',
  ' F'),
 ('most', ' [asthe rank well ]', ' [beautiful and compelling ]', ' F'),
 ('their', ' [out glances steal ]', ' [window at the ]', ' F'),
 ('sigh of relief', ' [a with up ]', ' [when something happens ]', ' F'),
 ('do things like',
  ' [They rareimperfections. the ]',
  ' [give up their 

In [276]:
dat[0][0]
cuewords = {'sorely':True, 'in':True, 'made':True, 'review':True, 'film':True, 'recommend':True,'needs':True,"showing":True, 'released':True, "The": True}
cuesymbols = {'*':True, '(':True, ')':True ,'"':True, "'":True, ':':True, 'II':True, 'III':True}

In [344]:
#Testing
#print(dat[1],dat[1][1])
#print(dat[1][1]+ dat[1][2], dat[1])
#(dat[1][1] +dat[1][2]).dtype
#isitcapitalized(dat[0])

AttributeError: 'list' object has no attribute 'dtype'

# Feature Generation

In [357]:
def isitcapitalized(testcase):
    """Is the label capitalized? Return True or False"""
    return testcase[1].isupper()


def isthereayear(testcase):
    """Is the YEAR (YYYY) format in the surrounding 3 words before and after? Return True or False"""
    combo = testcase[2] + testcase[3]
    joined = "".join(combo)
    yearsearch = re.compile(r"\d\d\d\d")
    if re.search(yearsearch, joined):
        return True
    else:
        return False
    
def hasCueWords(testcase,cueword=None):
    """Are any of the cuewords we determined found in the surrounding text? Return True or False"""
    combo = testcase[2] + testcase[3]
    #combo = combo.split(' ')
    if cueword:
        for word in combo:
            if word == cueword:
                return True
    else:
        for word in combo:
            if word in cuewords:
                return True
    return False

def hasCueSymbols(testcase,cuesymbol=None):
    """Are any of the cuesymbols we determined found in the surrounding text? Return True or False"""
    combo = testcase[2] + testcase[3]
    #combo = combo.split(' ')
    if cuesymbol:
        for symb in combo:
            if symb == cuesymbol:
                return True
    else:
        for symb in cuesymbols:
            return True
    return False

def hasAppostrophe(testcase):
    """Does "'s" (sign of film ownership) appear in the surrounding text? Return True or False"""
    combo = testcase[1] + testcase[2] + testcase[3] 
    for c in combo:
        if c.find("'s") >= 0:
            return True
    return False

def isPositive(testcase):
    """Is this a positive or negative example? Return True or False"""
    return testcase[-1]

def featureGen(cases):
    """This function brings together all of our feature generation functions and
    exports a pandas dataframe which can be fed into the ML classifiers"""
    #Set Key words and Key symbols
    cuewords = {'in':True, 'made':True, 'review':True, 'film':True, 'recommend':True, "showing":True, 'released':True, "The": True}
    cuesymbols = {'*':True, '(':True, ')':True ,'"':True, "'":True, ':':True, 'II':True, 'III':True}
    
    for cueword in cuewords:
        cuewords[cueword] = [hasCueWords(case, cueword) for case in cases]  
        
    for cuesymb in cuesymbols:
        cuesymbols[cuesymb] = [hasCueSymbols(case, cuesymb) for case in cases]   
        
    #Create subdataframes
    cuewords_df = pd.DataFrame(cuewords)
    cuesymbols_df = pd.DataFrame(cuesymbols)
    
    #Put it all together into master dataframe
    features = pd.DataFrame(
        {'hasYear':[isthereayear(case) for case in cases],
         'isCap':[isitcapitalized(case) for case in cases],
         'hasCueWords':[hasCueWords(case, cuewords) for case in cases],    
         #'hasApostrophe':[hasAppostrophe(case) for case in cases],
         'isPositive':[isPositive(case) for case in cases]
        }
    )
    df = pd.concat([cuewords_df, cuesymbols_df, features], axis=1)
    return(df)
    
    

In [57]:
#df = featureGen(cases)
#df

,The,film,in,made,recommend,released,review,showing,"""",',...,),*,:,II,III,hasApostrophe,hasCueWords,hasYear,isCap,isPositive
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,True,True
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,False,True,True
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,True
6,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,True,True
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
8,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,True,False,True
9,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,True,False,False


In [358]:
dframe = featureGen(dat)
dframe.head(n=100)

,The,film,in,made,recommend,released,review,showing,"""",',(,),*,:,II,III,hasCueWords,hasYear,isCap,isPositive
0,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True
1,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [320]:
#Clean DFRAME
#dframe['isPositive'] = dframe['isPositive'].replace([' T', ' F'], [True, False])
#dframe.isPositive.unique()

array(['True', 'False'], dtype=object)

In [334]:
dframe['isPositive'].dtype

dtype('bool')

# ML Models

In [377]:
def returnPrecision(y_truth, y_pred, classifier):
    """This function takes in an array of true observed and an array of predicted observed and outputs three 
    different measures for precision"""
    precision = {}
    precision["p_macro"] = precision_score(y_truth, y_pred, average='macro')
    precision["p_micro"] = precision_score(y_truth, y_pred, average='micro')
    precision["p_weight"] = precision_score(y_truth, y_pred, average='weighted')
    precision["p"] = precision_score(y_truth, y_pred, pos_label=True, average='binary')
    return pd.DataFrame(precision, index=[classifier])

def returnRecall(y_truth, y_pred, classifier):
    """This function takes in an array of true observed and an array of predicted observed and outputs three 
    different measures for precision"""
    recall = {}
    recall["r_macro"] = recall_score(y_truth, y_pred, average='macro')
    recall["r_micro"] = recall_score(y_truth, y_pred, average='micro')
    recall["r_weight"] = recall_score(y_truth, y_pred, average='weighted')
    recall["r"] = recall_score(y_truth, y_pred, pos_label=True, average="binary")
    return pd.DataFrame(recall, index=[classifier])

def returnF1(y_truth, y_pred, classifier):
    """This function takes in an array of true observed and an array of predicted observed and outputs three 
    different measures for F1"""
    F1 = {}
    F1["f1_macro"] = f1_score(y_truth, y_pred, average='macro')
    F1["f1_micro"] = f1_score(y_truth, y_pred, average='micro')
    F1["f1_weight"] = f1_score(y_truth, y_pred, average='weighted')
    F1["f1"] = f1_score(y_truth, y_pred, pos_label=True, average='binary')
    return pd.DataFrame(F1, index=[classifier])

def returnAccuracy(y_truth, y_pred, classifier):
    """This function takes in an array of true observed and an array of predicted observed and outputs accuracy"""
    accuracy = {}
    accuracy["accu"] = accuracy_score(y_truth, y_pred)
    return pd.DataFrame(accuracy, index=[classifier])

def returnMetrics(y_truth, y_pred, classifier):
    """Returns recall, precision, and accuracy as a pandas dataframe for each classifier"""
    rec = returnRecall(y_truth, y_pred, classifier)
    prec = returnPrecision(y_truth, y_pred, classifier)
    accu = returnAccuracy(y_truth, y_pred, classifier)
    f1 = returnF1(y_truth, y_pred, classifier)
    df = pd.concat([rec, prec, f1, accu], axis=1)
    return(df)

def runModels(X, y, kf, names, classifiers):
    """Function that runs over the classifiers and does k-fold cross-validation on the training set - returns metrics as dataframe"""
    df = pd.DataFrame([])
    for name, clf in zip(names, classifiers):
        for train_index, test_index in kf.split(X):
            clf.fit(X[train_index], y[train_index])
            predicted = clf.predict(X[test_index])
            data = returnMetrics(y[test_index], predicted, name)
            myscore = clf.score(X[test_index], y[test_index])
            score = pd.DataFrame({"score": myscore}, index = [name])
            ccat = pd.concat([data, score], axis = 1)
            df = df.append(ccat)
    return df

def runTruth(X, y, names, classifiers):
    """This function will run the ML classifiers on the testing set  - returns metrics as dataframe"""
    df = pd.DataFrame([])
    falses = []
    for name, clf in zip(names, classifiers):
        clf.fit(X, y)
        predicted = clf.predict(X)
        data = returnMetrics(y, predicted, name)
        df = df.append(data)
        for i in range(len(predicted)):
            if (predicted[i] != y[i]):
                outtuple = (i,predicted[i],y[i], name)
                falses.append(outtuple) 
    return (df, falses)
    

def MLmodels(X, Y, nsplits,crossv = True):
    """This function will run classifiers and export accuracy, precision, and recall for each classifier.
    N-fold cross validation has been automated here, where user determines number of folds"""
    ###########################################
    #Define classifiers
    ###########################################
    names = ["Decision Tree", "Logistic", "SVM", "Random Forest", "ExtraTrees"]
    classifiers = [ LogisticRegression(), tree.DecisionTreeClassifier(), svm.SVC(),
                   RandomForestClassifier(), ExtraTreesClassifier()]
    
    ###########################################
    #Create Train/Test Split
    ###########################################
    #X = df.ix[:, df.columns != 'isPositive'].values
    #y = np.ravel(df[['isPositive']].values)
    kf = KFold(n_splits = nsplits)
    
    ###########################################
    #Run Models
    ###########################################
    if(crossv == True): 
        return(runModels(X, y, kf, names, classifiers))
    else:
        return(runTruth(X, y, names, classifiers))
        
    
    

  

In [336]:
X = dframe.ix[:, dframe.columns != 'isPositive'].values
y = np.ravel(dframe[['isPositive']].values)

In [337]:
y

array([ True,  True,  True, ..., False, False, False], dtype=bool)

In [378]:
crossv = False
if crossv == True:
    result = MLmodels(X, y, 10,crossv)
else:
    (result,idlist) = MLmodels(X, y, 10,crossv)

In [363]:
#MLmodels(X, y, 10, crossv=False)

In [382]:
#idlist

In [383]:
for i in idlist:
    print(dat[i[0]], i[1], i[2], i[3])

(88, 'of a', ['lack', 'the', 'camouflage'], ['credible', 'story.', 'I'], False) True False Decision Tree
(295, 'a little', ['is', 'it', 'although'], ['more', '"bare', 'bones".'], False) True False Decision Tree
(358, 'one', ['success...Another', 'of', 'gauge'], ['of', 'my', 'cookie'], False) True False Decision Tree
(484, "iceskater's", ['Olympic', 'an', 'as'], ['sponsor.', 'The', 'skater,'], False) True False Decision Tree
(711, 'audience for', ['large', 'a', 'is'], ['this', 'movie', 'especially'], False) True False Decision Tree
(729, '10):', ['TO', '(0', 'BerardinelliRATING'], ['4.5Alternative', 'Scale:', '**'], False) True False Decision Tree
(887, 'time as does', ['screen', 'much', 'leastas'], ['Williams.', 'The', 'movie'], False) True False Decision Tree
(1015, 'of', ['command', 'his', 'while'], ['a', 'Federation', 'starship'], False) True False Decision Tree
(1247, 'agitatoduring the', ['furious', 'the', 'bits,'], ['fight', 'scenes,', 'war'], False) True False Decision Tree
(165

## Running on Set 2

After running initial k-fold CV on set 1, we have identified the baseline precision, recall, and F1. We are now ready to test on set 2. The goals of these next steps will be to a) explore false negatives and false positives in set 2. Based on this, we will modify our rules. Once we modify rules, we will re-run k-fold CV on set 1. If those results improve, we will then move on and finally test set 3. We will finally report precision, recall, and F1 for the third set as well as identify our best-performing classifier. 